In [2]:
import dash
from dash import dcc, html, Input, Output, ctx, State, ALL
import pandas as pd
import plotly.express as px

# Load the data
data_file = r'data_16b.csv'
df = pd.read_csv(data_file)

# Clean the data, handle inconsistencies with pricing
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle $3.00/lb cases
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  

df["Price"] = df.apply(clean_price, axis=1)

# Initialize the app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Grocery Price Comparison"

# Adding "All" as an option
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),

    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),

    html.Div([
        html.H3("Selected Products:", style={'text-align': 'center'}),
        html.Div([
            html.H4("Target:", style={'margin-top': '10px'}),
            html.Div(id="basket-target", style={'margin-bottom': '10px'}),
            html.H4("Total for Target: $", id="total-target", style={'margin-bottom': '10px'}),
        ], style={'border': '1px solid #8B0000', 'padding': '10px', 'margin-bottom': '20px'}),

        html.Div([
            html.H4("Whole Foods:", style={'margin-top': '10px'}),
            html.Div(id="basket-wholefoods", style={'margin-bottom': '20px'}),
            html.H4("Total for Whole Foods: $", id="total-wholefoods"),
        ], style={'border': '1px solid #BAFFC9', 'padding': '10px'}),

        html.Button("Clear Selection", id="clear-selection-button", n_clicks=0, 
                    style={'margin-top': '20px', 'background-color': '#f44336', 'color': 'white'}),
    ], style={'text-align': 'center'}),

    html.Div([
        dcc.Graph(id="cumulative-price-animation"),
    ], style={'margin-top': '30px'}),
])

# Store baskets for Target and Whole Foods
baskets = {"Target": [], "Whole Foods": []}

@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    [Input("category-dropdown", "value"),
     Input("product-search", "value")]
)
def search_products(search_clicks, category, product_name):
    # Filter data based on category and product name
    if not product_name:
        return "Please enter a product name."

    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False, na=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False, na=False))]

    if filtered_df.empty:
        return "No products found."

    # Ensure no N/A prices
    filtered_df = filtered_df.dropna(subset=["Price"])

    # Sort by lowest price
    filtered_df = filtered_df.sort_values(by="Price")

    def create_product_list(products, store):
        return html.Ul([
            html.Li(html.Button(f"{row['Name']} - ${row['Price']:.2f}",
                                id={"store": store, "name": row["Name"]}))
            for _, row in products.iterrows()
        ])

    target_products = filtered_df[filtered_df["Store"] == "Target"]
    wholefoods_products = filtered_df[filtered_df["Store"] == "Whole Foods"]

    return html.Div([
        html.Div([
            html.H3("Target:"),
            create_product_list(target_products, "Target"),
        ]),
        html.Div([
            html.H3("Whole Foods:"),
            create_product_list(wholefoods_products, "Whole Foods"),
        ])
    ])

@app.callback(
    [Output("basket-target", "children"),
     Output("basket-wholefoods", "children"),
     Output("total-target", "children"),
     Output("total-wholefoods", "children"),
     Output("cumulative-price-animation", "figure")],
    [Input({"store": "Target", "name": ALL}, "n_clicks"),
     Input({"store": "Whole Foods", "name": ALL}, "n_clicks")],
    [State({"store": "Target", "name": ALL}, "id"),
     State({"store": "Whole Foods", "name": ALL}, "id")]
)
def update_baskets(target_clicks, wholefoods_clicks, target_ids, wholefoods_ids):
    global baskets

    # Update Target basket
    for click, product_id in zip(target_clicks, target_ids):
        if click and not any(item["name"] == product_id["name"] for item in baskets["Target"]):
            product_name = product_id["name"]
            product_row = df[df["Name"] == product_name]
            if not product_row.empty:
                price = product_row.iloc[0]["Price"]
                baskets["Target"].append({"name": product_name, "price": price})

    # Update Whole Foods basket
    for click, product_id in zip(wholefoods_clicks, wholefoods_ids):
        if click and not any(item["name"] == product_id["name"] for item in baskets["Whole Foods"]):
            product_name = product_id["name"]
            product_row = df[df["Name"] == product_name]
            if not product_row.empty:
                price = product_row.iloc[0]["Price"]
                baskets["Whole Foods"].append({"name": product_name, "price": price})

    # Generate cumulative price animation
    all_items = []
    for store, items in baskets.items():
        for item in items:
            all_items.append({"Store": store, "Name": item["name"], "Price": item["price"]})

    # Ensure both stores appear even if no items are selected
    if not any(item["Store"] == "Target" for item in all_items):
        all_items.append({"Store": "Target", "Name": "No items selected", "Price": 0})
    if not any(item["Store"] == "Whole Foods" for item in all_items):
        all_items.append({"Store": "Whole Foods", "Name": "No items selected", "Price": 0})

    df_basket = pd.DataFrame(all_items)

    # Calculate cumulative prices
    df_basket["Cumulative Price"] = df_basket.groupby("Store")["Price"].cumsum()
    df_basket["Step"] = range(1, len(df_basket) + 1)  # Step for animation

    fig = px.bar(
        df_basket,
        x="Store",
        y="Cumulative Price",
        color="Store",
        animation_frame="Step",
        hover_data=["Name", "Price"],
        title="Cumulative Price Animation",
        labels={"Cumulative Price": "Cumulative Price ($)", "Store": "Store"},
    )

    fig.update_layout(
        xaxis_title="Store",
        yaxis_title="Cumulative Price ($)",
        legend_title="Store",
        height=600
    )

    # Return updated baskets and animation
    target_items = [f"{item['name']} - ${item['price']:.2f}" for item in baskets["Target"]]
    wholefoods_items = [f"{item['name']} - ${item['price']:.2f}" for item in baskets["Whole Foods"]]
    target_total = sum(item["price"] for item in baskets["Target"])
    wholefoods_total = sum(item["price"] for item in baskets["Whole Foods"])

    return (
        html.Ul([html.Li(item) for item in target_items]),
        html.Ul([html.Li(item) for item in wholefoods_items]),
        f"Total for Target: ${target_total:.2f}",
        f"Total for Whole Foods: ${wholefoods_total:.2f}",
        fig
    )